In [ ]:
import os
from glob import glob
import numpy as np
import gc

In [ ]:
!pip install -q torchsummaryX wandb --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sklearn.metrics
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from torch import optim
from torch.nn import init
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:",device)


from torchsummaryX import summary

from tqdm.notebook import tqdm

Device: cpu


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"adritadas","key":"4d8e5e73b5126973227f2e938ba1249f"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73274 sha256=691980e9ca4b5986864bb60f91b592a5e8b5d4ccbc07034d874b378b4f506118
  Stored in directory: /root/.cache/pip/wheels/f3/67/7b/a6d668747974998471d29b230e7221dd01330ac34faebe4af4
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!kaggle competitions download -c 11-785-s23-hw1p2
!mkdir '/content/data'

!unzip -qo '11-785-s23-hw1p2.zip' -d '/content/data'

100% 16.0G/16.0G [09:46<00:00, 28.4MB/s]
100% 16.0G/16.0G [09:46<00:00, 29.3MB/s]


In [ ]:
data_dir_dev_clean = "/content/data/11-785-s23-hw1p2/dev-clean"

data_dir_train_clean = "/content/data/11-785-s23-hw1p2/train-clean-100"

In [ ]:
train_X = sorted(glob(os.path.join(data_dir_train_clean, 'mfcc','*.npy')))
train_Y = sorted(glob(os.path.join(data_dir_train_clean, 'transcript','*.npy')))

val_X = sorted(glob(os.path.join(data_dir_dev_clean, 'mfcc', '*.npy')))
val_Y = sorted(glob(os.path.join(data_dir_dev_clean, 'transcript','*.npy')))

In [ ]:
root = "/content/data/11-785-s23-hw1p2/"
partition = "dev-clean"

In [ ]:
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

print(PHONEMES.index('[SIL]'))     

0


In [ ]:
torch.cuda.empty_cache()
gc.collect()


0

In [ ]:
mfcc_dir = sorted(glob(os.path.join(root, partition, 'mfcc','*.npy')))
transcript_dir = sorted(glob(os.path.join(root,partition,'transcript','*.npy')))


In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-360"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes
        
        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        #self.mfcc_dir       = 
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        #self.transcript_dir = 

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          =  sorted(glob(os.path.join(root, partition, 'mfcc','*.npy')))
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcript_names    = sorted(glob(os.path.join(root,partition,'transcript','*.npy')))

        

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names) 

        self.mfccs, self.transcripts = [], []

        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc        = np.load(mfcc_names[i])
        #   Do Cepstral Normalization of mfcc (explained in writeup)
        #   Load the corresponding transcript
            transcript  = np.load(transcript_names[i])[1:-1]  # Remove [SOS] and [EOS] from the transcript 
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        print(len(self.mfccs),len(self.transcripts))
      

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2) x 27, (T2+2) x 27 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that 
        # the final shape is T x 27 (Where T = T1 + T2 + ...) 
        self.mfccs          = np.concatenate(self.mfccs,axis=0)

        # TODO: Concatenate all transcripts in self.transcripts such that 
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts    = np.concatenate(self.transcripts,axis=0)
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done. 
        # self.mfcc is an array of the format (Frames x Features). 
        # Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. 
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs,((context,context),(0,0)), 'constant',constant_values=0)# TODO

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such. 
        # Hence, we map these phonemes to integers

        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        self.transcripts = [PHONEMES.index(self.transcripts[i]) for i in range (len(self.transcripts))]
        print("Length of mfccs:" ,len(self.mfccs),"Length of transcripts:",len(self.transcripts))
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        frames =  self.mfccs[ind:ind+2*self.context+1, :].reshape(-1)
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        

        frames      = torch.FloatTensor(frames) # Convert to tensors
        phonemes    = torch.tensor(self.transcripts[ind])       

        return frames, phonemes

In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-360"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes
        
        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        #self.mfcc_dir       = 
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        #self.transcript_dir = 

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          =  sorted(glob(os.path.join(root, partition, 'mfcc','*.npy')))
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        

        

        # Making sure that we have the same no. of mfcc and transcripts
         

        self.mfccs= []

        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc        = np.load(mfcc_names[i])
        #   Do Cepstral Normalization of mfcc (explained in writeup)
        #   Load the corresponding transcript
           # Remove [SOS] and [EOS] from the transcript 
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
          

        
          

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2) x 27, (T2+2) x 27 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that 
        # the final shape is T x 27 (Where T = T1 + T2 + ...) 
        self.mfccs          = np.concatenate(self.mfccs,axis=0)

        # TODO: Concatenate all transcripts in self.transcripts such that 
        # the final shape is (T,) meaning, each time step has one phoneme output
        
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done. 
        # self.mfcc is an array of the format (Frames x Features). 
        # Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. 
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs,((context,context),(0,0)), 'constant',constant_values=0)# TODO

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such. 
        # Hence, we map these phonemes to integers

        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        frames =  self.mfccs[ind:ind+2*self.context+1, :].reshape(-1)
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        

        frames      = torch.FloatTensor(frames) # Convert to tensors
               

        return frames

In [ ]:
config = {
    'epochs'        : 8,
    'batch_size'    : 1024*8,
    'context'       : 25,
    'init_lr'       : 1e-3,
    'architecture'  : 'unkown'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

In [ ]:
#TODO: Create a dataset object using the AudioDataset class for the training data 
train_data = AudioDataset(root, phonemes = PHONEMES, context=25, partition= "train-clean-100")

# TODO: Create a dataset object using the AudioDataset class for the validation data 
val_data = AudioDataset(root, phonemes = PHONEMES, context=25, partition= "dev-clean")

# TODO: Create a dataset object using the AudioTestDataset class for the test data 
test_data = AudioTestDataset(root, phonemes = PHONEMES, context=25, partition= "test-clean")

28539 28539
Length of mfccs: 36091207 Length of transcripts: 36091157
2703 2703
Length of mfccs: 1928254 Length of transcripts: 1928204


In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data, 
    num_workers = 4,
    batch_size  = config['batch_size'], 
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data, 
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data, 
    num_workers = 2, 
    batch_size  = config['batch_size'], 
    pin_memory  = True, 
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*27)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  8192
Context        :  25
Input size     :  1377
Output symbols :  42
Train dataset samples = 36091157, batches = 4406
Validation dataset samples = 1928204, batches = 236
Test dataset samples = 1934138, batches = 237


In [ ]:
for i, data in enumerate(train_loader):
  
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([8192, 1377]) torch.Size([8192])


In [ ]:
# Implement dropout,scheduler,kaiming
class TrainModel(torch.nn.Module):
     def __init__(self,in_features,out_features):
        super(TrainModel,self).__init__()

        self.model = torch.nn.Sequential(
            
            torch.nn.Linear(in_features , 1024),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(1024),
            torch.nn.Dropout(p=0.15),

            torch.nn.Linear(1024,1024),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(1024),
            torch.nn.Dropout(p=0.15),

            torch.nn.Linear(1024,2048),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(2048),
            torch.nn.Dropout(p=0.25),

            torch.nn.Linear(2048,2048),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(2048),
            torch.nn.Dropout(p=0.15),

            torch.nn.Linear(2048,2048),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(2048),
            torch.nn.Dropout(p=0.2),

            torch.nn.Linear(2048,2048),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(2048),
            torch.nn.Dropout(p=0.2),

            torch.nn.Linear(2048,2048),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(2048),
            torch.nn.Dropout(p=0.15),

            torch.nn.Linear(2048,1024),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(1024),
            torch.nn.Dropout(p=0.15),

            torch.nn.Linear(1024,out_features),
          
          )

     def forward(self,x):
       out = self.model(x)

       return out


In [ ]:
for i, data in enumerate(train_loader):
    frames, phonemes = data
    print(frames.shape, phonemes.shape)
    break

torch.Size([8192, 1377]) torch.Size([8192])


In [ ]:
in_features = (2*config['context'] + 1 ) * 27
out_features = len(train_data.phonemes)
model = TrainModel(in_features,out_features).to(device)
print(model)
summary(model,frames.to(device))

TrainModel(
  (model): Sequential(
    (0): Linear(in_features=1377, out_features=1024, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.15, inplace=False)
    (4): Linear(in_features=1024, out_features=1024, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.15, inplace=False)
    (8): Linear(in_features=1024, out_features=2048, bias=True)
    (9): ReLU()
    (10): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.25, inplace=False)
    (12): Linear(in_features=2048, out_features=2048, bias=True)
    (13): ReLU()
    (14): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): Dropout(p=0.15, inplace=False)
    (16): Linear(in_features=2048, out_features=2048, bias=True)
    (17): ReLU()
    (18): BatchNorm1d(2048, eps=1

/usr/local/lib/python3.8/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[1377, 1024]","[8192, 1024]",1411072.0,1410048.0
1_model.ReLU_1,-,"[8192, 1024]",NaN,NaN
2_model.BatchNorm1d_2,[1024],"[8192, 1024]",2048.0,1024.0
3_model.Dropout_3,-,"[8192, 1024]",NaN,NaN
4_model.Linear_4,"[1024, 1024]","[8192, 1024]",1049600.0,1048576.0
5_model.ReLU_5,-,"[8192, 1024]",NaN,NaN
6_model.BatchNorm1d_6,[1024],"[8192, 1024]",2048.0,1024.0
7_model.Dropout_7,-,"[8192, 1024]",NaN,NaN
8_model.Linear_8,"[1024, 2048]","[8192, 2048]",2099200.0,2097152.0


In [ ]:
criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr = config['init_lr'])


In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
def train(model, dataloader, optimizer,criterion):

  model.train()
  tloss,tacc = 0,0
  batch_bar = tqdm(total=len(train_loader),dynamic_ncols=True,leave=False,position=0,desc='Train')

  for i, (frames,phonemes) in enumerate(dataloader):

    optimizer.zero_grad()

    frames = frames.to(device)
    phonemes = phonemes.to(device)

    logits = model(frames)

    loss = criterion(logits,phonemes)

    loss.backward()

    optimizer.step()
    tloss += loss.item()
    tacc  += torch.sum(torch.argmax(logits, dim=1)==phonemes).item()/logits.shape[0]

    batch_bar.set_postfix(loss = "{:.04f}".format(float(tloss/(i+1))),
                           acc ="{:.04f}%".format(float(tacc*100/(i+1))))
    batch_bar.update()

    del frames,phonemes,logits
    torch.cuda.empty_cache()

  batch_bar.close()
  tloss /= len(train_loader)
  tacc /= len(train_loader)

  return tloss,tacc


In [ ]:
def eval(model, dataloader, optimizer,criterion):

  model.eval()
  vloss,vacc = 0,0
  batch_bar = tqdm(total=len(val_loader),dynamic_ncols=True,leave=False,position=0,desc='Train')

  for i, (frames,phonemes) in enumerate(dataloader):

   

    frames = frames.to(device)
    phonemes = phonemes.to(device)
    with torch.inference_mode():
      logits = model(frames)
      loss = criterion(logits,phonemes)

   

  
    vloss += loss.item()
    vacc  += torch.sum(torch.argmax(logits, dim=1)==phonemes).item()/logits.shape[0]

    batch_bar.set_postfix(loss = "{:.04f}".format(float(vloss/(i+1))),
                           acc ="{:.04f}%".format(float(vacc*100/(i+1))))
    batch_bar.update()

    del frames,phonemes,logits
    torch.cuda.empty_cache()

  batch_bar.close()
  vloss /= len(val_loader)
  vacc /= len(val_loader)

  return vloss,vacc


In [ ]:
torch.cuda.empty_cache()
gc.collect()

for epoch in range(config['epochs']):

  print("\nEpoch {}/{}".format(epoch+1,config['epochs']))

  curr_lr = float(optimizer.param_groups[0]['lr'])
  train_loss, train_acc = train(model,train_loader,optimizer,criterion)
  val_loss,val_acc = eval(model,val_loader,optimizer,criterion)

  print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss,curr_lr))
  print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100 , val_loss))

